# Uncertainty quantification using "DRIVE: Digital Retinal Images for Vessel Extraction" dataset
 
__author__: Yongchan Kwon

This note shows an example of the use of the method introduced in the paper "Uncertainty quantification using Bayesian neural networks in classification: Application to biomedical segmentation".

This notebook is based on the Walter de Back's amazing notebook https://gitlab.com/wdeback/dl-keras-tutorial/blob/master/notebooks/3-cnn-segment-retina-uncertainty.ipynb I really recommend to see together. 

**Reference**

- J.J. Staal, M.D. Abramoff, M. Niemeijer, M.A. Viergever, B. van Ginneken, "Ridge based vessel segmentation in color images of the retina", IEEE Transactions on Medical Imaging, 2004, vol. 23, pp. 501-509.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
np.random.seed(20190218)

from skimage.external import tifffile # read tiff images
from skimage.io import imread # read gif images
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
import seaborn as sns

from model import *
import utils
import time

plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
print('Backend: ', K.backend())
print('Image_data_format: ', K.image_data_format())
N_train = 200

Backend:  tensorflow
Image_data_format:  channels_last


In [3]:
# load training images
fns = sorted(glob.glob('./input/training/images/*.tif'))
x_train = np.array([tifffile.imread(fn) for fn in fns])

# load test images
fns = sorted(glob.glob('./input/test/images/*.tif'))
x_test = np.array([tifffile.imread(fn) for fn in fns])
print('shape of raw train data: ', x_train.shape)
print('shape of raw test data: ',x_test.shape)

/usr/local/lib/python3.5/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


shape of raw train data:  (20, 584, 565, 3)
shape of raw test data:  (20, 584, 565, 3)


In [4]:
 # load training annotations
fns = sorted(glob.glob('./input/training/1st_manual/*.gif'))
y_train = np.array([imread(fn) for fn in fns]) # read images
y_train = np.expand_dims(y_train, -1) # add channels dimension

# load test annotations
fns = sorted(glob.glob('./input/test/1st_manual/*.gif'))
y_test = np.array([imread(fn) for fn in fns]) # read images
y_test = np.expand_dims(y_test, -1) # add channels dimension
print('train shape:', y_train.shape)
print('test shape:', y_test.shape)

train shape: (20, 584, 565, 1)
test shape: (20, 584, 565, 1)


## Preprecossing

In [5]:
x_train = utils.preprocess(x_train)
x_test = utils.preprocess(x_test)

y_train = utils.preprocess(y_train)
y_test = utils.preprocess(y_test)

X_train, Y_train = utils.get_random_snippets(x_train, y_train, number=N_train, size=(96,96))
X_test, Y_test = utils.get_random_snippets(x_test, y_test, number=1000, size=(96,96))

min: 0.0, max: 1.0, shape: (20, 584, 565, 3), type: float32
min: 0.0, max: 1.0, shape: (20, 584, 565, 3), type: float32
min: 0.0, max: 1.0, shape: (20, 584, 565, 1), type: float32
min: 0.0, max: 1.0, shape: (20, 584, 565, 1), type: float32


In [6]:
batch_size = 32
epochs = 200

In [7]:
unet_16_model = UNet(N_filters=16)
unet_16_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', dice_coefficient, precision_smooth, recall_smooth])
print("Number of parameters: ", unet_16_model.count_params())
info_check_string='./weights/DRIVE_u_net_16.hdf5'
early_stopping=EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint=ModelCheckpoint(info_check_string, monitor='loss', save_best_only=True)

history = unet_16_model.fit(X_train, Y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      shuffle=True,
                      verbose=0,
                      validation_split=0.2, # 4 samples are used for a validation set
                      callbacks=[early_stopping, model_checkpoint]) 

score = unet_16_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss, acc, dice, precision, recall):', score)

Number of parameters:  1115265
1000/1000 [==============================] - 1s 1ms/step
Test loss, acc, dice, precision, recall): [0.1748147256374359, 0.9461942319869995, 0.6588937678337097, 0.6653704042434693, 0.6532777075767517]


In [8]:
unet_32_model = UNet(N_filters=32)
unet_32_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', dice_coefficient, precision_smooth, recall_smooth])
print("Number of parameters: ", unet_32_model.count_params())
info_check_string='./weights/DRIVE_u_net_32.hdf5'
early_stopping=EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint=ModelCheckpoint(info_check_string, monitor='loss', save_best_only=True)

history = unet_32_model.fit(X_train, Y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      shuffle=True,
                      verbose=0,
                      validation_split=0.2, # 4 samples are used for a validation set
                      callbacks=[early_stopping, model_checkpoint]) 

score = unet_32_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss, acc, dice, precision, recall):', score)

Number of parameters:  4452097
1000/1000 [==============================] - 2s 2ms/step
Test loss, acc, dice, precision, recall): [0.22372100043296814, 0.933634331703186, 0.5767761878967285, 0.7387910170555114, 0.4742603387832642]


In [9]:
small_16_model = Small_UNet(N_filters=16)
small_16_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', dice_coefficient, precision_smooth, recall_smooth])
print("Number of parameters: ", small_16_model.count_params())
info_check_string='./weights/DRIVE_200_small_16.hdf5'
early_stopping=EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint=ModelCheckpoint(info_check_string, monitor='loss', save_best_only=True)

history = small_16_model.fit(X_train, Y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      shuffle=True,
                      verbose=0,
                      validation_split=0.2, # 4 samples are used for a validation set
                      callbacks=[early_stopping, model_checkpoint]) 

score = small_16_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss, acc, dice, precision, recall):', score)

Number of parameters:  293761
1000/1000 [==============================] - 1s 966us/step
Test loss, acc, dice, precision, recall): [0.17731638133525848, 0.9382851600646973, 0.5758653607368469, 0.6546338076591491, 0.514663722038269]


In [10]:
small_32_model = Small_UNet(N_filters=32)
small_32_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', dice_coefficient, precision_smooth, recall_smooth])
print("Number of parameters: ", small_32_model.count_params())
info_check_string='./weights/DRIVE_200_small_32.hdf5'
early_stopping=EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint=ModelCheckpoint(info_check_string, monitor='loss', save_best_only=True)

history = small_32_model.fit(X_train, Y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      shuffle=True,
                      verbose=0,
                      validation_split=0.2, # 4 samples are used for a validation set
                      callbacks=[early_stopping, model_checkpoint]) 

score = small_32_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss, acc, dice, precision, recall):', score)

Number of parameters:  1170689
1000/1000 [==============================] - 2s 2ms/step
Test loss, acc, dice, precision, recall): [0.15524515116214752, 0.9477623677253724, 0.6738046631813049, 0.7166565594673157, 0.6366452074050903]


# Bayesian neural network: stochastic feed forward

In [11]:
# It takes a time !! 
num = len(X_test)
model_list = [unet_16_model, unet_32_model, small_16_model, small_32_model]
model_name_list = ['unet_16','unet_32','small_16','small_32']
result_dict = {}
for ind, model in enumerate(model_list):
    alea_list = []
    epis_list = []
    for i in range(num):
        image = X_test[i]
        gt    = Y_test[i]
        prediction, aleatoric, epistemic, scores = utils.predict(model, image, gt, T=5)
        alea_list.append(np.mean(aleatoric))
        epis_list.append(np.mean(epistemic))
    
    result_dict.update({model_name_list[ind] : 
    [np.mean(alea_list), np.std(alea_list),
     np.mean(epis_list), np.std(epis_list)]} )

In [12]:
(result_dict)

{'small_16': [0.036774598, 0.007652763, 0.0027609786, 0.0015513109],
 'small_32': [0.028372368, 0.008326541, 0.0026146132, 0.001440558],
 'unet_16': [0.03256932, 0.006708347, 0.004135161, 0.0020149916],
 'unet_32': [0.02214826, 0.0062294905, 0.0026109423, 0.0015459639]}

In [13]:
print("Number of parameters: ", unet_16_model.count_params())
print("Number of parameters: ", unet_32_model.count_params())
print("Number of parameters: ", small_16_model.count_params())
print("Number of parameters: ", small_32_model.count_params())

Number of parameters:  1115265
Number of parameters:  4452097
Number of parameters:  293761
Number of parameters:  1170689
